In [10]:
from Verifier_Agent import verifier_tool
from langgraph.graph import StateGraph, END,START
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage, BaseMessage, ToolMessage
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory, ChatVertexAI
from prompts import main_prompt
from langgraph.checkpoint.memory import MemorySaver

from typing import TypedDict, List
from Retriever.Retriever_Agent import retriever_agent
from Tools.Human_Response import human_response
from langchain.prompts import ChatPromptTemplate



# --- Model Configuration (Unchanged) ---
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}
model_kwargs = {
    "temperature": 0.28,
    "max_output_tokens": 1000,
    "top_p": 0.95,
    "top_k": None,
    "safety_settings": safety_settings,
}
model = ChatVertexAI(
    model_name="gemini-2.5-flash-lite",
    **model_kwargs
)


# --- Agent State Definition (Unchanged) ---
class AgentState(TypedDict):
    messages: List[BaseMessage]
    verified_results: str
    relevant_context: str
    condensed_query: str
    image_path: List[str]


def router_entry_node(state: AgentState) -> AgentState:
    """A simple node that acts as the starting point for the routing logic."""
    print("---ENTERING GRAPH, PREPARING TO ROUTE---")
    return state




def condense_query(state: AgentState) -> AgentState:
    """Condenses the chat history and latest user query into a standalone question."""
    print("---NODE: CONDENSING QUERY---")
    user_message = state["messages"][-1].content
    history = state["messages"][:-1]
    if not history:
        state["condensed_query"] = user_message
        print(f"---CONDENSED QUERY (No history): {user_message}---")
        return state

    formatted_history = "\n".join(

        [f"{'Human' if isinstance(msg, HumanMessage) else 'AI'}: {msg.content}" for msg in history])
    condensing_prompt = ChatPromptTemplate.from_messages([
        ("system",
         "Given the following conversation and a follow-up question, rephrase the follow-up question to be a standalone question, in its original language. Do not answer the question, just reformulate it."),
        ("human", "Chat History:\n{chat_history}\n\nFollow Up Input: {question}"),
    ])
    condenser_chain = condensing_prompt | model
    response = condenser_chain.invoke({"chat_history": formatted_history, "question": user_message})
    state["condensed_query"] = response.content
    print(f"---CONDENSED QUERY: {response.content}---")
    return state








In [11]:
memory = MemorySaver()


# --- 4. Graph Definition (Final Version) ---
graph = StateGraph(AgentState)

graph.add_node("router", router_entry_node)
graph.add_node("condense_query", condense_query)

graph.add_edge(START,'router')
graph.add_edge('router','condense_query')
graph.add_edge('condense_query',END)

app = graph.compile(checkpointer=memory)


In [12]:
from IPython.display import Image,display

display(Image(app.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`